# 🎛️ Лабораторная работа №3: Линейные системы и свёртка
**Курс:** Основы теории сигналов и систем (ОТСиС)  
**Цель:** Изучить реакцию линейных систем на входные сигналы и операцию свёртки.

## 🎯 Ключевые вопросы
1. Что такое LTI-система (Linear Time-Invariant)?
2. Что такое импульсная характеристика $h[n]$?
3. Как работает свёртка (Convolution)?
4. Как отфильтровать сигнал, зная $h[n]$?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
print('✅ Библиотеки готовы!')

## 1. Свёртка (Convolution)

Выход LTI-системы $y[n]$ — это свёртка входного сигнала $x[n]$ и импульсной характеристики $h[n]$:

$$ y[n] = x[n] * h[n] = \sum_{k=-\infty}^{\infty} x[k] h[n-k] $$

**Интуиция:** Мы "протаскиваем" перевернутую импульсную характеристику через сигнал.

In [ ]:
# Входной сигнал: прямоугольный импульс
x = np.array([0, 0, 1, 1, 1, 1, 0, 0])

# Импульсная характеристика: сглаживающий фильтр (Moving Average)
h = np.array([0.33, 0.33, 0.33])

# Вычисляем свёртку
y = signal.convolve(x, h, mode='full')

# Визуализация
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 10))

ax1.stem(x)
ax1.set_title('Вход x[n]')
ax1.set_ylim([0, 1.2])

ax2.stem(h)
ax2.set_title('Импульсная хар-ка h[n]')
ax2.set_xlim([0, len(x)])

ax3.stem(y)
ax3.set_title('Выход y[n] (Результат свёртки)')
ax3.set_xlabel('Отсчеты')

plt.tight_layout()
plt.show()

print(f"Длина входа: {len(x)}")
print(f"Длина ядра: {len(h)}")
print(f"Длина выхода: {len(y)} (L_x + L_h - 1)")

## 2. Импульсная характеристика (Impulse Response)

Это реакция системы на единичный импульс (удар). Зная $h[n]$, мы знаем о системе ВСЁ.

Примеры систем:
1. **Эхо:** $h = [1, 0, 0, 0.5]$ (сигнал повторяется с затуханием)
2. **Низкочастотный фильтр (Low Pass):** $h = [0.1, 0.1, ..., 0.1]$ (сглаживание)
3. **Высокочастотный фильтр (High Pass):** $h = [1, -1]$ (выделение перепадов)

In [ ]:
# Создадим искусственный звук (щелчок)
fs = 8000
t = np.arange(0, 0.5, 1/fs)
x_sound = np.zeros_like(t)
x_sound[100:200] = 1  # Короткий звук

# Создаем эффект ЭХО
# h = [1, 0, ..., 0, 0.6, 0, ..., 0, 0.3]
delay_samples = int(0.1 * fs) # 100 мс задержка
h_echo = np.zeros(delay_samples * 2 + 1)
h_echo[0] = 1.0           # Прямой звук
h_echo[delay_samples] = 0.6 # Первое эхо
h_echo[delay_samples*2] = 0.3 # Второе эхо

# Применяем эффект
y_echo = signal.convolve(x_sound, h_echo, mode='full')

# Визуализация
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(t, x_sound)
ax1.set_title('Исходный звук')
ax1.grid(True)

# Ось времени для выхода длиннее
t_out = np.arange(0, len(y_echo)) / fs
ax2.plot(t_out, y_echo, 'g')
ax2.set_title('Звук с эхом')
ax2.set_xlabel('Время, с')
ax2.grid(True)

plt.show()

## 🛠 ЗАДАНИЕ: Фильтрация зашумленного сигнала

**Дано:** Синусоида с сильным шумом.
**Задача:** Очистить сигнал, используя фильтр "Скользящее среднее" (Moving Average).

1. Сгенерируйте сигнал: $x(t) = \sin(2\pi \cdot 5 \cdot t) + 	ext{шум}$
2. Создайте фильтр $h$: массив из 50 элементов, равных $1/50$.
3. Примените свёртку `signal.convolve`.
4. Постройте на одном графике: исходный (серым) и очищенный (красным).

In [ ]:
# 1. Генерация
fs = 1000
t = np.arange(0, 2.0, 1/fs)
s_pure = np.sin(2*np.pi*5*t)
noise = 0.5 * np.random.randn(len(t))
x_noisy = s_pure + noise

# 2. Создание фильтра (Moving Average)
# Усредняем по 50 точкам (50 мс)
M = 50
h_ma = np.ones(M) / M

# 3. Фильтрация
# mode='same' чтобы длина выхода совпадала с входом
y_filtered = signal.convolve(x_noisy, h_ma, mode='same')

# 4. Визуализация
plt.figure(figsize=(12, 6))
plt.plot(t, x_noisy, 'gray', alpha=0.5, label='Зашумленный')
plt.plot(t, y_filtered, 'r', linewidth=2, label='Отфильтрованный (MA-50)')
plt.plot(t, s_pure, 'b--', linewidth=1, label='Идеал')
plt.title('Подавление шума методом скользящего среднего')
plt.legend()
plt.grid(True)
plt.show()